In [3]:
import tensorflow as tf
import time
import numpy as np

# Force reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Create synthetic image data (like CIFAR-10 but larger)
num_samples = 20000
img_height, img_width, channels = 64, 64, 3
num_classes = 10

X = np.random.rand(num_samples, img_height, img_width, channels).astype(np.float32)
y = np.random.randint(0, num_classes, size=(num_samples,))
y = tf.keras.utils.to_categorical(y, num_classes)

# Build a bigger CNN model
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (3,3), activation="relu", input_shape=(img_height, img_width, channels)),
        tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
        tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ])
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# Training function
def benchmark(device_name, epochs=3):
    with tf.device(device_name):
        model = create_model()
        start = time.time()
        history = model.fit(X, y, batch_size=128, epochs=epochs, verbose=0)
        end = time.time()
    print(f"{device_name} -> Time: {end-start:.2f}s, Final Acc: {history.history['accuracy'][-1]:.4f}")

# Run benchmarks
print("Running CPU vs GPU benchmark...\n")
benchmark("/CPU:0", epochs=3)
benchmark("/GPU:0", epochs=3)

Running CPU vs GPU benchmark...

/CPU:0 -> Time: 203.83s, Final Acc: 0.1020
/GPU:0 -> Time: 30.59s, Final Acc: 0.1003


Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


I0000 00:00:1755963518.312893  322746 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1755963518.312918  322746 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/opt/homebrew/Caskroom/miniforge/base/envs/tf_numpy2/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


/CPU:0 training time: 1.27 sec
/GPU:0 training time: 2.74 sec
